# GMSO - General Molecular Simulation Object

## Complex Atomtyping

[GMSO](https://github.com/mosdef-hub/gmso) allows for engine agnostic python object to store everything needed for writing molecule input files for simulation.


This notebook is designed to provide basics for interfacing with GMSO. Tutorials get more in-depth in terms of complex usage denoted by the `beginners`, `intermediates`, and `experts` tags on these notebooks. For a full list of the examples discussed in these tutorials, please see [the GMSO Tutorials README](README.md).

After completing this notebook, move on to [this notebook](04_for_intermediates.ipynb) to learn about how to write complex coarse-grained systems in `GMSO`.

Contained in this notebook is examples of:
* Typed vs untyped topologies
* Interactive `networkx`
* More complex writing

---

In [2]:
import gmso
from gmso.parameterization import apply

/Users/calcraven/miniconda3/envs/gmso-dev/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/test_mosdef_repos/mbuild/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]


## Typed vs untyped topologies
---

In [3]:
top = gmso.Topology.load("source_files/ethane.json")
top.is_typed()

False

In [4]:
ff = gmso.ForceField("source_files/ethane.xml")
ptop = apply(top, ff, identify_connections=True)
ptop.is_typed()

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:367: UserWarning: BondType between atoms CT and CT is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:367: UserWarning: BondType between atoms CT and HC is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:398: UserWarning: AngleType between atoms CT, CT and HC is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:398: UserWarning: AngleType between atoms HC, CT and HC is missing from the ForceField
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/core/forcefield.py:459: UserWarning: DihedralType between atoms HC, CT, CT and HC is missing from the ForceField.
  warnin

True

In [7]:
ptop.save("source_files/typed-ethane.json", overwrite=True)

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/gmso/gmso/formats/json.py:49: UserWarning: The provided topology is typed and `types` is set to False. The types(potentials) info will be lost in the serialized representation. Please consider using `types=True` if this behavior is not intended. 
  warnings.warn(


## Interactive networkx
---

In [ ]:
top = gmso.Topology.load("source_files/ethane.json")
top.is_typed()
ff = gmso.ForceField("source_files/ethane.xml")
ptop = apply(
    top, ff, identify_connections=True, 
    ignore_params=["site", "bond", "angle", "dihedral", "improper"],
    remove_untyped=False
)

In [ ]:
from gmso.formats.networkx import interactive_networkx_atomtypes
interactive_networkx_atomtypes(ptop)

## More complex writing
---

In [ ]:
import mbuild as mb
from pdbfixer import PDBFixer 
from openmm.app import PDBFile

from gmso.external import from_mbuild, from_parmed
! wget -O source_files/3O49.pdb https://files.rcsb.org/view/3O49.pdb

fixer = PDBFixer(filename="source_files/3O49.pdb")
fixer.addMissingHydrogens(7.0)
PDBFile.writeFile(fixer.topology, fixer.positions, open("saved_files/protein.pdb", 'w'))
protein = mb.load("saved_files/protein.pdb")
children_to_remove = set()
for particle in protein.particles():
    if "HOH" in particle.parent.name:
        children_to_remove.add(particle)
protein.remove(children_to_remove)

water = mb.load("O", smiles=True)
water.name="water"
cpd = mb.fill_box([protein, water], [1,1000], box=[10,20,10])


top = cpd.to_gmso()
top

In [ ]:
waterXml = gmso.ForceField("source_files/tip3p.xml")
waterXml.combining_rule = "geometric"
proteinXml = gmso.ForceField("oplsaa")
proteinXml = gmso.ForceField("../../../gmso/gmso/utils/files/gmso_xmls/test_ffstyles/gaff.xml")

molecules = top.unique_site_labels(name_only=True)
ffDict = {}
for mol in molecules:
    if mol == "water":
        ffDict[mol] = waterXml
    else:
        ffDict[mol] = proteinXml

parameterizedTop = apply(
    top, ffDict, identify_connections=True,
    ignore_params=["site", "bond", "angle", "dihedral", "improper"],
    remove_untyped=False,
)

In [ ]:
parameterizedTop.is_fully_typed()

In [ ]:
missing_types = parameterizedTop.get_untyped("topology")
for missing in missing_types:
    print(f"Missing types for {missing}: {len(missing_types[missing])}")